In [1]:
# Block 1: Load Train Dataset
import face_recognition
import os

train_dir = 'dataset/train'
known_encodings = []
known_names = []

print("[INFO] Encoding faces from train/ folder...")

for person_name in os.listdir(train_dir):
    person_path = os.path.join(train_dir, person_name)
    if not os.path.isdir(person_path):
        continue

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        image = face_recognition.load_image_file(img_path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            known_encodings.append(encodings[0])
            known_names.append(person_name)

print(f"[INFO] Loaded {len(known_encodings)} face encodings from train/ dataset.")


[INFO] Encoding faces from train/ folder...
[INFO] Loaded 4125 face encodings from train/ dataset.


In [2]:
import cv2
from datetime import datetime
import csv

attendance = {}

# Initialize webcam
video = cv2.VideoCapture(0)
if not video.isOpened():
    print("Error: Could not open video stream.")
    exit()

print("[INFO] Starting attendance system... Press 'q' to quit")

while True:
    ret, frame = video.read()
    if not ret:
        break

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for encoding, location in zip(face_encodings, face_locations):
        distances = face_recognition.face_distance(known_encodings, encoding)
        min_distance = min(distances) if len(distances) > 0 else 1.0

        if min_distance < 0.45:
            match_index = distances.tolist().index(min_distance)
            name = known_names[match_index]
        else:
            name = "Unknown"

        # Do not save attendance for Unknown faces
        if name != "Unknown" and name not in attendance:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            attendance[name] = timestamp
            print(f"{name} marked present at {timestamp}")
        
        # Draw box and name (label unknown faces as 'Unknown')
        top, right, bottom, left = [v * 4 for v in location]
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    cv2.imshow("AI Attendance System", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

# Save attendance (only for recognized faces, no unknown faces)
with open("attendance.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Name", "Timestamp"])
    for name, timestamp in attendance.items():
        writer.writerow([name, timestamp])

print("✅ Attendance saved to attendance.csv")


[INFO] Starting attendance system... Press 'q' to quit
Keshav Bansal marked present at 2025-05-08 09:52:23
Bhavya Mittal marked present at 2025-05-08 09:52:34
Dharmpreet marked present at 2025-05-08 09:52:56
✅ Attendance saved to attendance.csv


In [ ]:
# Block 3: Capture unknown faces, ask for name, and split into train/test

import os
import cv2
import face_recognition
from datetime import datetime
import random

# Parameters
dataset_train_dir = "dataset/train"
dataset_test_dir = "dataset/test"
os.makedirs(dataset_train_dir, exist_ok=True)
os.makedirs(dataset_test_dir, exist_ok=True)

max_unknown_images = 100
saved_count = 0
tolerance = 0.5  # for matching

# Start webcam
video = cv2.VideoCapture(0)
if not video.isOpened():
    print("Error: Could not open webcam.")
    exit()

print(" Capturing faces... Press 'q' to stop manually.")

# Ask for the name once
person_name = input("Enter the name for the captured faces (or type 'Unknown'): ").strip()

# Create folders if not exist
train_person_dir = os.path.join(dataset_train_dir, person_name)
test_person_dir = os.path.join(dataset_test_dir, person_name)
os.makedirs(train_person_dir, exist_ok=True)
os.makedirs(test_person_dir, exist_ok=True)

while saved_count < max_unknown_images:
    ret, frame = video.read()
    if not ret:
        break

    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for encoding, location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_encodings, encoding, tolerance=tolerance)

        if not any(matches):
            top, right, bottom, left = [coord * 4 for coord in location]
            face_img = frame[top:bottom, left:right]

            if face_img.size > 0:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                filename = f"{person_name}_{saved_count + 1}_{timestamp}.jpg"

                # Decide randomly: 80% Train, 20% Test
                if random.random() < 0.8:
                    save_path = os.path.join(train_person_dir, filename)
                else:
                    save_path = os.path.join(test_person_dir, filename)

                cv2.imwrite(save_path, face_img)
                print(f"📸 Saved: {save_path}")
                saved_count += 1

                # Draw box
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                cv2.putText(frame, f"{person_name} {saved_count}", (left, top - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

                if saved_count >= max_unknown_images:
                    break

    cv2.imshow("Capturing Faces", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("🛑 Stopped manually.")
        break

video.release()
cv2.destroyAllWindows()

print(f"✅ Finished. {saved_count} images saved for '{person_name}' into Train/Test folders!")


🔍 Capturing faces... Press 'q' to stop manually.
🛑 Stopped manually.
✅ Finished. 0 images saved for 'abc' into Train/Test folders!


In [3]:
# Block 4: Accuracy Checker (including Unknown faces)
import os
import face_recognition

test_dir = 'dataset/test'
total_tests = 0
correct_predictions = 0
unknown_correct = 0
unknown_total = 0

print("[INFO] Testing faces from test/ folder...")

for person_name in os.listdir(test_dir):
    person_path = os.path.join(test_dir, person_name)
    if not os.path.isdir(person_path):
        continue

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        image = face_recognition.load_image_file(img_path)
        encodings = face_recognition.face_encodings(image)

        if not encodings:
            continue

        total_tests += 1
        encoding = encodings[0]

        distances = face_recognition.face_distance(known_encodings, encoding)
        min_distance = min(distances) if len(distances) > 0 else 1.0

        if min_distance < 0.45:
            match_index = distances.tolist().index(min_distance)
            predicted_name = known_names[match_index]
        else:
            predicted_name = "Unknown"

        if person_name == "Unknown":
            unknown_total += 1
            if predicted_name == "Unknown":
                unknown_correct += 1

        if predicted_name == person_name:
            correct_predictions += 1
        else:
            print(f"Wrong: Expected '{person_name}', but got '{predicted_name}' for image {img_name}")


accuracy = (correct_predictions / total_tests) * 100 if total_tests > 0 else 0
unknown_accuracy = (unknown_correct / unknown_total) * 100 if unknown_total > 0 else 0

print(f"\nAccuracy (known faces): {accuracy:.2f}% ({correct_predictions}/{total_tests} correct)")
print(f"Unknown Accuracy: {unknown_accuracy:.2f}% ({unknown_correct}/{unknown_total} correct)")


[INFO] Testing faces from test/ folder...
Wrong: Expected 'pins_Adriana Lima', but got 'pins_camila mendes' for image Adriana Lima109_11.jpg
Wrong: Expected 'pins_Adriana Lima', but got 'Unknown' for image Adriana Lima111_14.jpg
Wrong: Expected 'pins_Adriana Lima', but got 'Unknown' for image Adriana Lima113_16.jpg
Wrong: Expected 'pins_Adriana Lima', but got 'Unknown' for image Adriana Lima116_19.jpg
Wrong: Expected 'pins_Alex Lawther', but got 'Unknown' for image Alex Lawther148_36.jpg
Wrong: Expected 'pins_Alex Lawther', but got 'Unknown' for image Alex Lawther148_37.jpg
Wrong: Expected 'pins_Alex Lawther', but got 'Unknown' for image Alex Lawther148_38.jpg
Wrong: Expected 'pins_Alex Lawther', but got 'Unknown' for image Alex Lawther149_39.jpg
Wrong: Expected 'pins_Alvaro Morte', but got 'Unknown' for image Alvaro Morte141_178.jpg
Wrong: Expected 'pins_alycia dabnem carey', but got 'Unknown' for image alycia dabnem carey115_17.jpg
Wrong: Expected 'pins_Amanda Crew', but got 'Unknown